In [ ]:
!pip install /kaggle/input/kerasapplications -q
!pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps

In [ ]:
import os, gc

import efficientnet.tfkeras as efn
import numpy as np
import pandas as pd
import tensorflow as tf

Unhide below to see helper functions:

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy


def build_decoder(with_labels=True, target_size=(300, 300), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3)
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3)
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0
        img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
BATCH_SIZE = strategy.num_replicas_in_sync * 16

In [ ]:
model_paths = [
    '../input/ranzcr-last-models/0.952_model_640_47.h5',
    '../input/ranzcr-last-models/0.953_model_616_51.h5',
    '../input/ranzcr-last-models/0.953_model_640_43.h5',
    '../input/ranzcr-last-models/0.954_model_640_42.h5',
    '../input/ranzcr-last-models/0.954_model_632_48.h5',
]

subs = []
for model_path in model_paths:
    load_dir = f"/kaggle/input/{COMPETITION_NAME}/"
    sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
    test_paths = load_dir + "test/" + sub_df['StudyInstanceUID'] + '.jpg'

    label_cols = sub_df.columns[1:]

    image_size = int(model_path.split('_')[2])
    test_decoder = build_decoder(with_labels=False, target_size=(image_size, image_size))
    dtest = build_dataset(
        test_paths, bsize=BATCH_SIZE, repeat=False, 
        shuffle=False, augment=False, cache=False,
        decode_fn=test_decoder
    )
    with strategy.scope():
        model = tf.keras.models.load_model(model_path)

    print('predict:', model_path)
        
    sub_df[label_cols] = model.predict(dtest, verbose=1)
    subs.append(sub_df)
    
    del model
    gc.collect()

In [ ]:
submission = pd.concat(subs)
submission = submission.groupby('StudyInstanceUID').mean()
submission.to_csv('submission.csv')
submission